In [1]:
from tree_sitter import Language,Parser
GO_LANGUAGE = Language('build/go.so', 'go')
JS_LANGUAGE = Language('build/javascript.so', 'javascript')
PY_LANGUAGE = Language('build/python.so', 'python')
parser = Parser()
parser.set_language(PY_LANGUAGE)
# tree = parser.parse(bytes("""
# def foo():
#     if bar:
#         baz()""", "utf8"))
tree = parser.parse(bytes("""
class BootyBayBodyguard(MinionCard):
    def __init__(self):
        super().__init__("Booty Bay Bodyguard", 5, CHARACTER_CLASS.ALL, CARD_RARITY.COMMON)

    def create_minion(self, player):
        return Minion(5, 4, taunt=True)""", "utf8"))


In [11]:
cursor.node.get('children')

AttributeError: 'tree_sitter.Node' object has no attribute 'get'

In [2]:
root_node = tree.root_node
assert root_node.type == 'module'
assert root_node.start_point == (1, 0)
assert root_node.end_point == (3, 13)
print(root_node.end_point)
function_node = root_node.children[0]
assert function_node.type == 'function_definition'
assert function_node.child_by_field_name('name').type == 'identifier'

function_name_node = function_node.children[1]
assert function_name_node.type == 'identifier'
assert function_name_node.start_point == (1, 4)
assert function_name_node.end_point == (1, 7)

assert root_node.sexp() == "(module "\
    "(function_definition "\
        "name: (identifier) "\
        "parameters: (parameters) "\
        "body: (block "\
            "(if_statement "\
                "condition: (identifier) "\
                "consequence: (block "\
                    "(expression_statement (call "\
                        "function: (identifier) "\
                        "arguments: (argument_list))))))))"\


(3, 13)


In [2]:
cursor = tree.walk()


In [3]:
[x for x in cursor.__dir__() if x[:2]!='__']

['current_field_name',
 'goto_parent',
 'goto_first_child',
 'goto_next_sibling',
 'node']

In [3]:
print(cursor.node)
print(cursor.goto_first_child())
print(cursor.goto_next_sibling())
print(cursor.goto_parent())
print(cursor.current_field_name())
print(cursor.node.child_count)

<Node kind=module, start_point=(1, 0), end_point=(6, 39)>
True
False
True
None
1


In [5]:
attr = [x for x in cursor.node.__dir__() if x[:2]!='__']
#attr = [x for x in cursor.node.__dir__() ]
#getattr(cursor.node,attr[1])()
attr

['walk',
 'sexp',
 'child_by_field_id',
 'child_by_field_name',
 'type',
 'is_named',
 'is_missing',
 'has_changes',
 'has_error',
 'start_byte',
 'end_byte',
 'start_point',
 'end_point',
 'children',
 'child_count',
 'named_child_count',
 'next_sibling',
 'prev_sibling',
 'next_named_sibling',
 'prev_named_sibling',
 'parent']

In [3]:
print(tree.root_node.children[0].children[1])
print(tree.root_node.named_child_count)


<Node kind=identifier, start_point=(1, 6), end_point=(1, 23)>
1


In [4]:
from treelib import Tree,Node
tree_clone = Tree()
tree_id = 0
node_id_list = list()
tree_clone.create_node(tag=tree.root_node.type,identifier=tree_id,data=tree.root_node)
node_id_list.append([tree.root_node,tree_id])
print(node_id_list)
print(type(node_id_list[0][0]))
print(1)
tree_clone.show()

[[<Node kind=module, start_point=(1, 0), end_point=(6, 39)>, 0]]
<class 'tree_sitter.Node'>
1
module



In [5]:
from collections import deque

todo = deque([tree.root_node])
while todo:
    node = todo.popleft()
    print(node)
    # print(node)
    # print(tree.root_node)
    # print(node.__hash__)
    # print(id(tree.root_node))
    # print(tree.root_node==node)
    if node.child_count:
        todo.extend(node.children)
        #print(todo)
        for i in node.children:
            tree_id+=1
            node_id_list.append([i,tree_id])
            #print([i,tree_id])
            temp = [ j for j in range(len(node_id_list))if node_id_list[j][0]==i.parent]
            #print(temp)
            #print(node_id_list[temp[0]][0])
            #print(len(temp)==1)
            tree_clone.create_node(tag=i.type,identifier=tree_id,data=i,parent=node_id_list[temp[0]][1])
            
tree_clone.show()

<Node kind=module, start_point=(1, 0), end_point=(6, 39)>
<Node kind=class_definition, start_point=(1, 0), end_point=(6, 39)>
<Node kind="class", start_point=(1, 0), end_point=(1, 5)>
<Node kind=identifier, start_point=(1, 6), end_point=(1, 23)>
<Node kind=argument_list, start_point=(1, 23), end_point=(1, 35)>
<Node kind=":", start_point=(1, 35), end_point=(1, 36)>
<Node kind=block, start_point=(2, 4), end_point=(6, 39)>
<Node kind="(", start_point=(1, 23), end_point=(1, 24)>
<Node kind=identifier, start_point=(1, 24), end_point=(1, 34)>
<Node kind=")", start_point=(1, 34), end_point=(1, 35)>
<Node kind=function_definition, start_point=(2, 4), end_point=(3, 91)>
<Node kind=function_definition, start_point=(5, 4), end_point=(6, 39)>
<Node kind="def", start_point=(2, 4), end_point=(2, 7)>
<Node kind=identifier, start_point=(2, 8), end_point=(2, 16)>
<Node kind=parameters, start_point=(2, 16), end_point=(2, 22)>
<Node kind=":", start_point=(2, 22), end_point=(2, 23)>
<Node kind=block, sta

In [7]:
import os
tree_clone.to_graphviz(filename='tree.gv', shape='ellipse')
os.system('dot -Tpng tree.gv -o tree.gv1.png')
os.startfile('tree.gv1.png')


FileNotFoundError: [WinError 2] 系统找不到指定的文件。: 'tree.gv1.png'

In [38]:
def make_move(cursor, move, all_nodes):
    # 递归遍历该树，把每个节点的信息保存起来，包括结点的类型、涉及范围的代码行起始位置、终止位置。
    # cursor: 即当前光标的位置（即节点的位置），通过cursor.node即可获取当前节点
    # move: 把move参数作为当前节点的移动方向
    # all_nodes: 保存节点信息的列表 （保存的是前序遍历的结果：根左右）

    if (move == "down"):
        all_nodes.append(cursor.node)
        if (cursor.goto_first_child()):
            make_move(cursor, "down", all_nodes)
        elif (cursor.goto_next_sibling()):
            make_move(cursor, "right", all_nodes)
        elif (cursor.goto_parent()):
            make_move(cursor, "up", all_nodes)
    elif (move == "right"):
        all_nodes.append(cursor.node)
        if (cursor.goto_first_child()):
            make_move(cursor, "down", all_nodes)
        elif (cursor.goto_next_sibling()):
            make_move(cursor, "right", all_nodes)
        elif (cursor.goto_parent()):
            make_move(cursor, "up", all_nodes)
    elif move == "up":
        if (cursor.goto_next_sibling()):
            make_move(cursor, "right", all_nodes)
        elif (cursor.goto_parent()):
            make_move(cursor, "up", all_nodes)

cursor = tree.walk()
all_nodes = []
make_move(cursor, "down", all_nodes)
print(all_nodes)


[<Node kind=module, start_point=(1, 0), end_point=(3, 13)>, <Node kind=function_definition, start_point=(1, 0), end_point=(3, 13)>, <Node kind="def", start_point=(1, 0), end_point=(1, 3)>, <Node kind=identifier, start_point=(1, 4), end_point=(1, 7)>, <Node kind=parameters, start_point=(1, 7), end_point=(1, 9)>, <Node kind="(", start_point=(1, 7), end_point=(1, 8)>, <Node kind=")", start_point=(1, 8), end_point=(1, 9)>, <Node kind=":", start_point=(1, 9), end_point=(1, 10)>, <Node kind=block, start_point=(2, 4), end_point=(3, 13)>, <Node kind=if_statement, start_point=(2, 4), end_point=(3, 13)>, <Node kind="if", start_point=(2, 4), end_point=(2, 6)>, <Node kind=identifier, start_point=(2, 7), end_point=(2, 10)>, <Node kind=":", start_point=(2, 10), end_point=(2, 11)>, <Node kind=block, start_point=(3, 8), end_point=(3, 13)>, <Node kind=expression_statement, start_point=(3, 8), end_point=(3, 13)>, <Node kind=call, start_point=(3, 8), end_point=(3, 13)>, <Node kind=identifier, start_point